## Library Imports

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix

import itertools
import os
import shutil
import random
from time import time
import matplotlib.pyplot as plt
#import pillow as pil
%matplotlib inline

In [18]:
DATA_PATH = 'SampleData'
NUM_LAYERS_TO_REMOVE = 5
NUM_LAYERS_TO_TRAIN = 20
LAYERS_TO_UNFREEZE = 3

## Create image pre-processing function

In [3]:
# Function to preprocess the image with Keras
preprocess_input = tf.keras.applications.mobilenet_v3.preprocess_input
#def image_preprocessing(file):
#    img = image.load_img(file, target_size=(224, 224))
#    img_array = image.img_to_array(img)
#    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
#    return tf.keras.applications.mobilenet_v3.preprocess_input(img_array_expanded_dims)

## Load the metadata file.  Create a ground truth column.

In [4]:
# Load the metadata file
metadata = pd.read_excel('metadata_modified.xlsx')
image_path = metadata['Image Path']
# Create the ground truth values and put it into a new 'g_truth' column
g_truth = metadata['Malignant'] + metadata['A']*2 + metadata['F']*4 + metadata['PT']*8 + metadata['TA']*16 + metadata['DC']*32 + metadata['LC']*64 + metadata['MC']*128 + metadata['PC']*256
metadata['g_truth'] = g_truth

FileNotFoundError: [Errno 2] No such file or directory: 'metadata_modified.xlsx'

## Display a test image

In [5]:
# Display test image
from IPython.display import Image
Image(filename=image_path[0], width=300,height=200)

NameError: name 'image_path' is not defined

In [6]:
mobile_v3 = tf.keras.applications.MobileNetV3Large()
#preprocessed_image = image_preprocessing(image_path[7117])
#predictions = mobile_v3.predict(preprocessed_image)
#print("Output shape: ", predictions.shape)
#results = imagenet_utils.decode_predictions(predictions)
#results

Remove the specifiied number of layers.
Freeze the specified number of layers.

In [8]:
# Remove last layers
#x = mobile_v3.layers[-NUM_LAYERS_TO_REMOVE].output
#output = Dense(units=2, activation='softmax')(x)

# Freeze layers
#for layer in new_model.layers[:-NUM_LAYERS_TO_TRAIN]:
#    layer.trainable = False                            

## Define the training and validation datasets

In [7]:
path_to_files = 'BreaKHis_v1/BreaKHis_v1/histology_slides/breast/'
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  path_to_files,
  validation_split=0.25,
  subset="training",
  seed=42,
  image_size=(224, 224),
#  batch_size=32,
  shuffle=True
)
  
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  path_to_files,
  validation_split=0.25,
  subset="validation",
  seed=42,
  image_size=(224, 224),
#  batch_size=32,
  shuffle=True
)
#val_batches = tf.data.experimental.cardinality(val_dataset)
#val_dataset = val_dataset.take((2*val_batches) // 5)
#test_dataset = val_dataset.skip((2*val_batches) // 5)


Found 7909 files belonging to 8 classes.
Using 5932 files for training.
Found 7909 files belonging to 8 classes.
Using 1977 files for validation.


## Use AUTOTUNE to decrease I/O roadblocks

In [8]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
valid_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)
#test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

## Define the model

In [9]:
IMG_SIZE=(224,224)
IMG_SHAPE = IMG_SIZE + (3,)
def new_model (image_shape=IMG_SIZE):
    input_shape = image_shape + (3,)
    base_model = tf.keras.applications.MobileNetV3Large(input_shape=IMG_SHAPE,
                                                        include_top=False, # important
                                                        weights='imagenet')
    # freeze the base model
    base_model.trainable = False
    # create input layer
    inputs = tf.keras.Input(shape=input_shape)
    # pre-process inputs
    x = preprocess_input(inputs)
    # set training to False to avoid tracking statistics in batch norm layer
    x = base_model(x, training=False)
    # add new binary classification layers
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    # include dropout with probability of 0.2 to avoid overfitting
    x = tf.keras.layers.Dropout(0.2)(x)

    # user prediction layer of one neuron (binary classifier only needs one - multiclass may need more)
    prediction_layer = tf.keras.layers.Dense(1)
    outputs = prediction_layer(x)

    model = tf.keras.Model(inputs,outputs)
    return model

In [16]:
IMG_SIZE=(224,224)
IMG_SHAPE = IMG_SIZE + (3,)
def new_model2 (image_shape=IMG_SIZE):
    input_shape = image_shape + (3,)
    base_model = tf.keras.applications.MobileNetV3Large(input_shape=IMG_SHAPE,
                                                        include_top=False, # important
                                                        weights='imagenet')
    
    # freeze the base model
    base_model.trainable = False

    # unfreeze some layers
    for layer in base_model.layers[-LAYERS_TO_UNFREEZE:]:
        layer.trainable = True

    # create input layer
    inputs = tf.keras.Input(shape=input_shape)
    # pre-process inputs
    x = preprocess_input(inputs)
    # set training to False to avoid tracking statistics in batch norm layer
    x = base_model(x, training=False)
    # add flatten layer
    x = tf.keras.layers.Flatten()(x)
    #x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dense(8, activation='relu')(x)

    # include dropout with probability of 0.2 to avoid overfitting
    #x = tf.keras.layers.Dropout(0.2)(x)

    # user prediction layer of one neuron (binary classifier only needs one - multiclass may need more)
#    prediction_layer = tf.keras.layers.Dense(1)
#    outputs = prediction_layer(x)
    outputs = x

    model = tf.keras.Model(inputs,outputs)
    return model

## Train the model

In [19]:
project5_model = new_model2(IMG_SIZE)
base_learning_rate = 0.01
opt = tf.keras.optimizers.legacy.SGD(learning_rate=base_learning_rate, momentum=0.9)
#project5_model.compile(optimizer=opt,
#            loss=tf.keras.losses.MeanAbsoluteError(),
#            metrics=["accuracy"])
project5_model.compile(optimizer='adam',
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])

initial_epochs = 10
history = project5_model.fit(
    train_dataset, 
    epochs=initial_epochs,
    validation_data=val_dataset)
project5_model.summary()

Epoch 1/10
186/186 [==============================] - 128s 668ms/step - loss: 1.8100 - accuracy: 0.3321 - val_loss: 1.6045 - val_accuracy: 0.3971
Epoch 2/10
186/186 [==============================] - 119s 633ms/step - loss: 1.5253 - accuracy: 0.4132 - val_loss: 1.7029 - val_accuracy: 0.4360
Epoch 3/10
186/186 [==============================] - 117s 625ms/step - loss: 1.3760 - accuracy: 0.4420 - val_loss: 1.5991 - val_accuracy: 0.4057
Epoch 4/10
186/186 [==============================] - 113s 602ms/step - loss: 1.2908 - accuracy: 0.4862 - val_loss: 1.8617 - val_accuracy: 0.4593
Epoch 5/10
186/186 [==============================] - 109s 584ms/step - loss: 1.1980 - accuracy: 0.5367 - val_loss: 1.7688 - val_accuracy: 0.5078
Epoch 6/10
186/186 [==============================] - 118s 629ms/step - loss: 1.1197 - accuracy: 0.5486 - val_loss: 1.6956 - val_accuracy: 0.4932
Epoch 7/10
186/186 [==============================] - 119s 636ms/step - loss: 1.0792 - accuracy: 0.5563 - val_loss: 1.6554 -

In [15]:
loss0, accuracy0 = project5_model.evaluate(val_dataset)

print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

62/62 [==============================] - 30s 444ms/step - loss: 1.5528 - accuracy: 0.5225
initial loss: 1.55
initial accuracy: 0.52
